### 8.1 Table Definition

#### 8.1.1 Review of Table as Column Dictionary

In [1]:
t:flip `name`iq!(`Dent`Beeblebrox`Prefect;98 42 126)
t
t[2;]    / extract row
t[;`iq]  / extract column
t[`iq]   / extract column works as well without semicolon
t[2;`iq] / extract value
type t   / 98h

name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126


name| `Prefect
iq  | 126


98 42 126


98 42 126


126


98h


- A table is logically a list of section dictionaries.
- The only effect of flipping the column dictionary is to reverse the order of its indices; no data is rearranged under the covers.

#### 8.1.3 Table-Definition Syntax

In [2]:
t:([] name:`Dent`Beeblebrox`Prefect; iq:98 42 126)
t
t~flip `name`iq!(`Dent`Beeblebrox`Prefect;98 42 126)

name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126


1b


In [3]:
t1:([] name:`Dent`Beeblebrox`Prefect; iq:98 42 126; who:`alien) / some columns can be atoms, but at least one should be list
t1
flip `c1`c2`c3!(`a`b`c;42;1.1)

name       iq  who  
--------------------
Dent       98  alien
Beeblebrox 42  alien
Prefect    126 alien


c1 c2 c3 
---------
a  42 1.1
b  42 1.1
c  42 1.1


The value lists in table-definition syntax can originate from variables, which is useful for programmatic table definition. In this case the column names are the variable names.

In [4]:
c1:`Dent`Beeblebrox`Prefect
c2:98 42 126
c:([] c1; c2)
c

c1         c2 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126


In [5]:
([] c1:1+til 5; c2:5#42) / any valid q expression can appear in columns definitions

c1 c2
-----
1  42
2  42
3  42
4  42
5  42


In [6]:
([] enlist `a; c2:100)    / single row table


x c2 
-----
a 100


#### 8.1.4 Table Metadata

In [7]:
cols t

`name`iq


In [8]:
meta t

c   | t f a
----| -----
name| s    
iq  | j    


- The key column c of the result contains the column names.
- The column t contains a symbol denoting the type char of the column.
- The column f contains the domains of any foreign key or link columns.
- The column a contains any attributes associated with the column.

When meta displays an upper-case type char for a column, this indicates that column is a compound list in which all fields are simple lists of the indicated type. Such tables arise, for example, when you group without aggregating in a query. Here is one created manually. Observe the upper case 'J' in the t column for column c2.

In [9]:
meta ([] c1:1 2 3; c2:(1 2; enlist 3; 4 5 6))

c | t f a
--| -----
c1| j    
c2| J    


In [9]:
tables `.   / list of tables in symbolic namespace ( root context here)
\a          / same as above

`c`t`t1


[0;31m         / same as above[0m: [0;31m         / same as above[0m

#### 8.1.5 Records

In [10]:
count t

3


In [11]:
t[1]   / record dictionary

name| `Beeblebrox
iq  | 42


In [12]:
value t[1]   / to retrieve just values, without column names

`Beeblebrox
42


#### 8.1.6 Flipped Column Dictionary vs. List of Records

Is a table a flipped column dictionary or a list of records? Logically it is both, but physically it is stored as a column dictionary. In fact, **q dynamically recognizes a conforming list of dictionaries that could be formed into a table and reorganizes the data into columnar form automatically**. It doesn't ask for permission or seek forgiveness.

In [13]:
type (`name`iq!(`Dent;98); `nome`iq!(`Beeblebrox;42))   / not a table as keys are not conforming
type (`name`iq!(`Dent;98); `name`iq!(`Beeblebrox;42))   / table as keys are conforming

0h


98h


In [14]:
p:enlist `a`b!10 20
p
type p  / p is a table with 1 record

a  b 
-----
10 20


98h


### 8.2 Empty Tables and Schema

Fully listing columns with literals is usually done only with smaller tables – e.g., lookup tables. Large tables are usually created programmatically from computed data or data read from files or received over the wire.

In these circumstances, it is useful to create an empty table initially and then populate it later by appending in place. You could do this with general empty lists.

A downside of tables being stored as columns is that **row deletion is an expensive operation because all the column lists must be compressed to close the resulting gap**. The best way to deal with this in large tables is not to do it. Instead of deleting a row, use a separate column that holds a flag indicating whether the row has been deleted and then exclude the "deleted" rows. Then compress this column to save space since it will be sparse

In [15]:
([] name:(); iq:())

name iq
-------


It is good practice to specify the types of all columns in an empty table. In the table definition, cast an empty list to the appropriate type.

In [16]:
([] name:`symbol$(); iq:`int$())

name iq
-------


Shorter technique, but less readable

In [17]:
([] name:0#`; iq:0#0)~ ([] name:`symbol$(); iq:`long$())

1b


### 8.3 Basic select and update

In [18]:
select from t

name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126


In [19]:
select name from t

name      
----------
Dent      
Beeblebrox
Prefect   


In [20]:
select newname:name,newiq:iq from t

newname    newiq
----------------
Dent       98   
Beeblebrox 42   
Prefect    126  


The syntax of update is the same as select, but named columns represent replacement by the values to the right of the colon. In our example,

In [21]:
update iq:iq%100,iq1:iq+100 from t / iq - existing column, iq1 - new column

name       iq   iq1
-------------------
Dent       0.98 198
Beeblebrox 0.42 142
Prefect    1.26 226


### 8.4 Primary Keys and Keyed Tables

#### 8.4.1 Keyed Table

The approach is to place the key column in a separate table parallel to a table containing the remaining columns to obtain a table of keys and a table of values. 

**A keyed table is a dictionary mapping a table of key records to a table of value records.** This represents a mapping from each row in a table of (presumably unique) keys to a corresponding row in a table of values – i.e., a positional correspondence of key rows to value rows. 

A keyed table is not a table – it is a dictionary and so has type 99h.

Keys should be unique but (sadly) this is not enforced. As we have already noted, dictionary creation does not enforce key uniqueness. A value row associated with a duplicate key is not accessible via key lookup, but it can be retrieved via a select on the key column.

In [22]:
v:flip `name`iq!(`Dent`Beeblebrox`Prefect;98 42 126) / values table
v
k:flip (enlist `eid)!enlist 1001 1002 1003           / keys table
k
kt:k!v                                               / keyed table
kt

name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126


eid 
----
1001
1002
1003


eid | name       iq 
----| --------------
1001| Dent       98 
1002| Beeblebrox 42 
1003| Prefect    126


#### 8.4.3 Keyed-Table Definition Syntax

In [23]:
/ flipped dictionaries form
kt:(flip (enlist `eid)!enlist 1001 1002 1003)!flip `name`iq!(`Dent`Beeblebrox`Prefect;98 42 126)  

In [24]:
/ table syntax
kt:([eid:1001 1002 1003] name:`Dent`Beeblebrox`Prefect; iq:98 42 126)
kt

eid | name       iq 
----| --------------
1001| Dent       98 
1002| Beeblebrox 42 
1003| Prefect    126


In [25]:
ktempty:([eid:()] name:(); iq:())  / empty keyed table
ktempty

eid| name iq
---| -------


#### 8.4.4 Accessing Records of a Keyed Table

In [26]:
kt[(enlist `eid)!enlist 1002]  / horrible
kt[1002]                       / better

name| `Beeblebrox
iq  | 42


name| `Beeblebrox
iq  | 42


In [27]:
kt[1002][`iq]

42


After the customary moment of q Zen, we realize that the net effect of "placing a key on a table" is to convert indexing of the rows from row number to key value.

#### 8.4.5 Retrieving Multiple Records

In [28]:
/kt 1001 1002 / not working
kt[(enlist 1001;enlist 1002)]
kt[flip enlist 1001 1002]
kt ([] eid:1001 1002)
([] eid:1001 1002)#kt

name       iq
-------------
Dent       98
Beeblebrox 42


name       iq
-------------
Dent       98
Beeblebrox 42


name       iq
-------------
Dent       98
Beeblebrox 42


eid | name       iq
----| -------------
1001| Dent       98
1002| Beeblebrox 42


#### 8.4.6 Reverse Lookup

In [29]:
kts:([eid:1001 1002 1003] name:`Dent`Beeblebrox`Prefect)
kts
kts?([] name:`Prefect`Dent)

eid | name      
----| ----------
1001| Dent      
1002| Beeblebrox
1003| Prefect   


eid 
----
1003
1001


#### 8.4.7 Components of a Keyed Table

Since a keyed table is a dictionary mapping the table of keys to the table of values, the functions **key** and **value** extract the constituents.

In [30]:
key kt    / key table
value kt  / values table

eid 
----
1001
1002
1003


name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126


In [31]:
keys kt   / list of key columns
cols kt   / list of all columns 

,`eid


`eid`name`iq


#### 8.4.8 Tables vs. Keyed Tables

It is possible to convert dynamically between a regular table having a column of potential key values and the corresponding keyed table using dyadic primitive **xkey**. The right operand is the source table/keyed table and the left operand is a symbol (or list of symbols) with the column name(s) to be used as the key.

In [32]:
t:([] eid:1001 1002 1003; name:`Dent`Beeblebrox`Prefect; iq:98 42 126)
t

eid  name       iq 
-------------------
1001 Dent       98 
1002 Beeblebrox 42 
1003 Prefect    126


In [33]:
`eid xkey t

eid | name       iq 
----| --------------
1001| Dent       98 
1002| Beeblebrox 42 
1003| Prefect    126


Conversely, to convert a keyed table to a regular table, use xkey with an empty general list as the left operand.

In [34]:
kt:([eid:1001 1002 1003] name:`Dent`Beeblebrox`Prefect; iq:98 42 126)
() xkey kt

eid  name       iq 
-------------------
1001 Dent       98 
1002 Beeblebrox 42 
1003 Prefect    126


You can also use an overload of ! to key/unkey tables. The left operand is a non-negative integer that specifies the number of left-most columns to include in the key, where 0 indicates none – i.e., no keys. With t and kt as above,
While these forms are terse, the first makes your code less obvious since the new key column(s) are only implicit.

In [35]:
1!t
0!kt

eid | name       iq 
----| --------------
1001| Dent       98 
1002| Beeblebrox 42 
1003| Prefect    126


eid  name       iq 
-------------------
1001 Dent       98 
1002 Beeblebrox 42 
1003 Prefect    126


If xkey is applied with a column that does not contain unique values, the result is not a error but rather a keyed table that does not have a true primary key.

In [36]:
t:([] eid:1001 1002 1003 1001; name:`Dent`Beeblebrox`Prefect`Dup)
ktdup:`eid xkey t
ktdup

eid | name      
----| ----------
1001| Dent      
1002| Beeblebrox
1003| Prefect   
1001| Dup       


Duplicate key values are not accessible via key lookup.

#### 8.4.9 Compound Primary Key

In [37]:
ktc:([lname:`Dent`Beeblebrox`Prefect; fname:`Arthur`Zaphod`Ford]; iq:98 42 126)
ktc

lname      fname | iq 
-----------------| ---
Dent       Arthur| 98 
Beeblebrox Zaphod| 42 
Prefect    Ford  | 126


Lookup

In [38]:
ktc[`lname`fname!`Beeblebrox`Zaphod]
ktc[`Dent`Arthur]

iq| 42


iq| 98


#### 8.4.10 Retrieving Records with a Compound Primary Key

Unlike a simple key, we can lookup multiple value records with a list of compound keys.

In [39]:
ktc (`Dent`Arthur; `Prefect`Ford)

iq 
---
98 
126


Of course the nifty construct with an anonymous table works with compound keys too.

In [40]:
ktc ([] lname:`Dent`Prefect; fname:`Arthur`Ford)

iq 
---
98 
126


As does the use of # to retrieve a sub keyed table from a list of keys.

In [41]:
K:([] lname:`Dent`Prefect; fname:`Arthur`Ford)
K#ktc

lname   fname | iq 
--------------| ---
Dent    Arthur| 98 
Prefect Ford  | 126


#### 8.4.11 Extracting Column Data

In [42]:
kts:([k:101 102 103] v1:`a`b`c; v2:1.1 2.2 3.3)
ktc:([k1:`a`b`c;k2:`x`y`z] v1:`a`b`c; v2:1.1 2.2 3.3)

In [43]:
kts[([] k:101 103)][`v1]
ktc[([] k1:`a`c;k2:`x`z)]
ktc[([] k1:`a`c;k2:`x`z)][`v1`v2]

`a`c


v1 v2 
------
a  1.1
c  3.3


a   c  
1.1 3.3


indexing-at-depth

In [44]:
kts[([] k:101 103); `v1]
ktc[([] k1:`a`c;k2:`x`z)]
ktc[([] k1:`a`c;k2:`x`z); `v1`v2]

`a`c


v1 v2 
------
a  1.1
c  3.3


`a 1.1
`c 3.3


### 8.5 Foreign Keys and Virtual Columns

#### 8.5.1 Definition of Foreign Key`

**A foreign key** is one or more table columns whose values are defined as an enumeration over the key column(s) of a keyed table. As in the case of symbol enumeration `sym$, the enumeration restricts foreign key values to be in the list of primary key values.

#### 8.5.2 Example of Simple Foreign Key

In [45]:
kt:([eid:1001 1002 1003] name:`Dent`Beeblebrox`Prefect; iq:98 42 126)


This table has no restriction on eid other than it be a long. To ensure that only eid values for actual travelers can be entered, we make the eid column in this table a foreign key related to the eid column in kt. This is done by enumerating over the name of the keyed table – i.e., **`kt$.**

When q sees the name of a keyed table in an enumeration domain it knows to use the list of key records.

In [46]:
([] eid:1003 1001 1002 1001 1002 1001; sc:126 36 92 39 98 42)

eid  sc 
--------
1003 126
1001 36 
1002 92 
1001 39 
1002 98 
1001 42 


As in the case of symbol enumeration, q looks up the index of each foreign key value in the list of key records and, under the covers, replaces the field value with that index. Also as with symbols, the enumeration is displayed in reconstituted form instead of as the underlying indices. To see the underlying indices, cast to an integer.

Attempting to enumerate a value that is not in the primary key column causes an error.

We put this together with table-definition syntax to define a details table with a foreign key over kt.

In [47]:
tdetails:([] eid:`kt$1003 1001 1002 1001 1002 1001; sc:126 36 92 39 98 42)
tdetails

eid  sc 
--------
1003 126
1001 36 
1002 92 
1001 39 
1002 98 
1001 42 


Observe that a foreign key is denoted by the name of the target keyed table in the f column in the output of meta.


In [48]:
meta tdetails

c  | t f  a
---| ------
eid| j kt  
sc | j     


The built-in function **fkeys** applied to a table (or keyed table) returns a dictionary in which each foreign key column name is mapped to its primary key table name.

In [49]:
fkeys tdetails

eid| kt


#### 8.5.3 Resolving a Foreign Key

When you wish to resolve a foreign key – i.e., get the actual values instead of enumerated values – apply **value** to the enumerated column.

In [50]:
tdetails1:update value eid from tdetails
tdetails1

eid  sc 
--------
1003 126
1001 36 
1002 92 
1001 39 
1002 98 
1001 42 


#### 8.5.4 Foreign Keys and Relations

Let tf be a table having a foreign key column f enumerated over a keyed table kt. All columns in kt are available via dot notation in a select expression whose from domain is tf. To access a column c in kt, use the notation f.c in the select expression. This column takes the name c by default in the result.

In [51]:
select eid.name, sc from tdetails

name       sc 
--------------
Prefect    126
Dent       36 
Beeblebrox 92 
Dent       39 
Beeblebrox 98 
Dent       42 


In [51]:
There is an implicit left join between tdetails and kt here.

[0;31mhere.[0m: [0;31mhere.[0m

The implicit join with dot notation is powerful and convenient when your tables are in normal form and there are multiple foreign key relations. For example, a query could retrieve

select name.street.city.zip.country from residents where …

in a single select with no explicit joins.

### 8.6 Working with Tables and Keyed Tables

In [52]:
t:([] name:`Dent`Beeblebrox`Prefect; iq:98 42 126)
kt:([eid:1001 1002 1003] name:`Dent`Beeblebrox`Prefect; iq:98 42 126)

#### 8.6.1 Appending Records (,:)

The fundamental way to append a record to a table is to view the table as a list of records and join with **,:**. Note that the fields in the record do not need to be in column order.

In [53]:
t,:`name`iq!(`W; 26)
t,:`iq`name!(200; `Albert)
t

name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126
W          26 
Albert     200


In [54]:
t,:(`H; 142) / if names are ignored than values should be in order of columns
t

name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126
W          26 
Albert     200
H          142


#### 8.6.2 First and Last Records

Because a table is logically a list of records, the functions first and last retrieve the initial and final records, respectively.

In [55]:
first t
last t

name| `Dent
iq  | 98


name| `H
iq  | 142


A keyed table is a dictionary so functions apply to the value table.

In [56]:
first kt
last kt

name| `Dent
iq  | 98


name| `Prefect
iq  | 126


You can retrieve the **first or last n records of a table or keyed table using the take operator #**. Why does this work? Tables are lists and keyed tables are dictionaries and # works on both. Since # always returns a list and the extracted records conform, q recognizes the result of # as a table or keyed table with the same schema as the input.

In [57]:
2#t

name       iq
-------------
Dent       98
Beeblebrox 42


In [58]:
-3#kt

eid | name       iq 
----| --------------
1001| Dent       98 
1002| Beeblebrox 42 
1003| Prefect    126


#### 8.6.3 Find

The find operator ? used with a table returns the index of a record – i.e., its row number.

In [59]:
t?`name`iq!(`Dent;98) / all columns should be used
t?(`Dent;98) / can be abbreviated, provided values are in right type and order
t?((`Dent;98);(`Prefect;126)) / find atomic

0


0


0 2


Since a keyed table is a dictionary, find ? performs a reverse lookup of a value record/row and returns the first associated key record.

In [60]:
kt?`name`iq!(`Dent;98)
kt?(`Dent;98)

eid| 1001


eid| 1001


As with key lookup, a single column must have enlisted values, or you can use the anonymous table construct.

In [61]:
t1:([] eid:1001 1002 1003)
t1?enlist each 1001 1002
t1?([] eid:1001 1002)

0 1


0 1


#### 8.6.4 Union with ,

The join operator , is defined for tables and keyed tables since they both comprise lists of records. It is essentially the same as UNION in SQL.

You can use , to append a record to (a copy of) a table, but no type checking will be performed.

In [62]:
t,`name`iq!(`Slaartibartfast; `123)

name            iq  
--------------------
Dent            98  
Beeblebrox      42  
Prefect         126 
W               26  
Albert          200 
H               142 
Slaartibartfast `123


Tables having exactly the same meta result can be joined to form a table. Since a table is a list of records, the result is obtained by appending the records of the right operand to those of the left.

In [63]:
t,([] name:1#`W; iq:1#26)

name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126
W          26 
Albert     200
H          142
W          26 


In [64]:
t,t

name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126
W          26 
Albert     200
H          142
Dent       98 
Beeblebrox 42 
Prefect    126
W          26 
Albert     200
H          142


Two keyed tables with the same meta result can be joined with ,. Because a keyed table is a dictionary whose keys and values are record lists, the operation has upsert semantics. Keys in the right operand that are not in the left operand are treated as append (i.e., insert), whereas the right operand acts as an update on common key values. In other words, the right operand is upserted into the left.

In [65]:
kt,([eid:1003 1004] name:`Prefect`W; iq:150 26)

eid | name       iq 
----| --------------
1001| Dent       98 
1002| Beeblebrox 42 
1003| Prefect    150
1004| W          26 


#### 8.6.5 Coalesce ^

**Coalesce ^** can be used to merge two keyed tables having the same columns. Its behavior derives from its behavior on dictionaries. For a common key value and a common column, the value of the column in the right keyed table prevails over that of column in the left keyed table, except where the right column is null, in which case the left column value survives. On non-common keys the individual values carry thru.

The behavior of ^ is the same as , when there are no nulls in a column in the right table.

In [66]:
([k:`a`b`c] v:10 0N 30)^([k:`a`b`c] v:100 200 0N)
([k:`a`b`c`x] v:10 0N 30 40)^([k:`a`b`c`y]; v:100 200 0N 0N)

k| v  
-| ---
a| 100
b| 200
c| 30 


k| v  
-| ---
a| 100
b| 200
c| 30 
x| 40 
y|    


The performance of ^ is slower than that of , since fields of the right operand must be checked for null.

#### 8.6.6 Column Join

Two tables with the same number of records can be joined sideways with join-each (,') to create a column join in which the columns are aligned in parallel

In [67]:
([] c1:`a`b`c),'([] c2:100 200 300)

c1 c2 
------
a  100
b  200
c  300


When the column lists of the tables are not disjoint ( have same columns ), the operation on the common columns has upsert semantics because each record is a dictionary.

In [68]:
([] c1:`a`b`c; c2:1 2 3),'([] c2:100 200 300)

c1 c2 
------
a  100
b  200
c  300


A sideways join on keyed tables requires that the key records conform, meaning that the key columns must have identical meta. The columns from the right operand are aligned along common keys and appended elsewhere.

In [69]:
([k:1 2 3] v1:10 20 30),'([k:3 4 5] v2:1000 2000 3000)

k| v1 v2  
-| -------
1| 10     
2| 20     
3| 30 1000
4|    2000
5|    3000


### 8.8 Attributes

Attributes are metadata that you attach to lists of special forms. They are also used on a dictionary domain or a table column to speed retrieval for some operations. The q interpreter can make certain optimizations based on the structure of the list implied by the attribute.

Attributes (other than `g#) are descriptive rather than prescriptive. By this we mean that by applying an attribute you are asserting that the list has a special form, which q will check. It does not instruct q to (re)make the list into the special form; that is your job. 

The attribute is applied to the target list in place, not on a copy.

A list operation that respects the form specified by the attribute leaves the attribute intact (other than `p#), while an operation that breaks the form results in the attribute being removed in the result.

#### 8.8.1 Sorted `s#

Applying the sorted attribute `s# to a simple list indicates that the items of the list are sorted in ascending order; there is no way to indicate a descending sort. When a list has the sorted attribute, linear search is replaced with binary search, which makes certain operations faster – for example, find ?, equality =, in and within.

In [70]:
`s#1 2 4 8
/ `s#2 1 4 8 / error s-fail
asc 2 1 8 4 / automatically applies `s#

`s#1 2 4 8


`s#1 2 4 8


In [75]:
L:`s#1 2 3 4 5
L,:6  / `s# still valid
L
L,:0  /  `s# not valid anymore
L

`s#1 2 3 4 5 6


1 2 3 4 5 6 0


**Tip** One place to apply the sorted attribute is on the date or time column of a simple time series.

Applying the sorted attribute to a table unintuitively applies the parted attribute (see next section) to the first column.

Applying the sorted attribute to a dictionary applies the attribute to the key list. Lookup is faster because binary search is used. A side effect of the way binary search is implemented is that the mapping given by the dictionary is now a step function, meaning that values between successive key values are "filled in."

#### 8.8.2 Unique `u#

Applying the unique attribute `u# to a list indicates that the items of the list are distinct. 

Applying `u# essentially causes q to create a hash table, which uses storage and adds overhead. It is best to apply the attribute after the list has been created, if possible. If you have a column that will always have unique values and your large table does not change often, you can get a significant performance speedup with `u#.

In [76]:
`u#2 1 4 8

`u#2 1 4 8


In [76]:
/ `u#2 1 4 8 2 - fails

[0;31mu-fail[0m: [0;31mu-fail[0m

In [79]:
L:`u#2 1 4 8
L
L,:3  / attribute preserved
L
L,:1  / attribute lost
L

`u#2 1 4 8


`u#2 1 4 8 3


2 1 4 8 3 1


#### 8.8.3 Parted `p#

The parted attribute `p# indicates that all common occurrences of any value in a list are adjacent. Viewing the list as a map, the parted attribute says that its graph is a step function with distinct steps. The parted attribute can be applied to a simple list of any type whose underlying value is integral – e.g., the integer types, dates, times, timestamps, timespans. You can also apply parted on a list of enumerated symbols since the underlying index values are integral.

The parted attribute causes q to create an underlying structure that keeps track of the steps. Because the attribute is not preserved under most operations you should apply it only after the list is fully created. One exception is that when two lists have `p# and their values are disjoint, the attribute will be preserved when the lists are joined with ,.

In [80]:
`p#2 2 2 1 1 4 4 4 4 3 3

`p#2 2 2 1 1 4 4 4 4 3 3


In [80]:
/ `p#2 2 2 1 1 4 4 4 4 3 3 2  / fails

[0;31mu-fail[0m: [0;31mu-fail[0m

With the exception in the note above, the parted attribute is not preserved under any operation on the list, even if the operation preserves the property.

In [83]:
L:`p#1 1 2 3 3
L
L,:3
L

`p#1 1 2 3 3


1 1 2 3 3 3


#### 8.8.4 Grouped `g#

The grouped attribute `g# differs from other attributes in that it can be applied to any list. It causes q to create and maintain an index – essentially a hash table. Grouped can be applied to a list when no other assumptions about its structure can be made.

In [84]:
`g#1 2 3 2 3 4 3 4 5 2 3 4 5 4 3 5 6

`g#1 2 3 2 3 4 3 4 5 2 3 4 5 4 3 5 6


The grouped attribute is maintained as operations are performed on the list, which can cause significant processing overhead in addition to the storage required. Best to apply it after the entire list has been created, if possible.

Applying the grouped attribute to a table column roughly corresponds to placing an index on a column in an RDBMS. 

#### 8.8.5 Remove Attribute `#

The operations `# removes any attribute that may currently be applied

In [85]:
L:`s#til 10
L

`s#0 1 2 3 4 5 6 7 8 9


In [86]:
`#L

0 1 2 3 4 5 6 7 8 9
